## Testing

In [1]:
import sys
!{sys.executable} -m pip install opencv-python IPython ffmpeg tqdm google-generativeai google cv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: /Users/shy/Projects/clippy/env/bin/python -m pip install --upgrade pip


In [2]:
from pathlib import Path
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
import cv2
import shutil
import ffmpeg
import subprocess
import os
from tqdm import tqdm

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

/Users/shy/Projects/clippy/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [4]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [5]:
model = genai.GenerativeModel('gemini-pro')

In [6]:
response = model.generate_content("What is the meaning of life?")

In [7]:
print(response)
to_markdown(response.text)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'finish_reason': 4, 'index': 0, 'safety_ratings': [], 'token_count': 0, 'grounding_attributions': []}]}),
)


ValueError: The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.

In [ ]:
response.prompt_feedback

In [ ]:
response.candidates


In [ ]:
response = model.generate_content("hi man how are you", stream=True)

## Pro-Vision Video!

In [8]:
model = genai.GenerativeModel('gemini-pro-vision')


In [9]:
!ls /Users/shy/Desktop/4-19-Sasha-blocked.MP4   

/Users/shy/Desktop/4-19-Sasha-blocked.MP4


In [10]:
FRAME_EXTRACTION_DIRECTORY = "frames"
FRAME_PREFIX = "_frame"

video_file_name = Path("/Users/shy/Desktop/4-19-Sasha-blocked.MP4")
video_file_name = Path("/Users/shy/Desktop/4-19-Sasha-dinner.MP4")
video_file_name = Path("/Users/shy/Desktop/test_clip.mp4")
video_file_name = Path("/Users/shy/Desktop/intros.mp4")

audio_file_name = Path(f"{FRAME_EXTRACTION_DIRECTORY}/{video_file_name.stem}.mp3")

def call_ffmpeg(arg_string):
    subprocess.call(['ffmpeg', "-hide_banner", "-loglevel", "error", "-y"] + arg_string.split())
    
def extract_mp3_from_mp4(video_file_name, audio_file_name):
    call_ffmpeg(f"-i {video_file_name} {audio_file_name}")

def convert_to_mp4(video_file_name):
    mkv_name = video_file_name.with_suffix(".mkv")
    call_ffmpeg(f"-i {video_file_name} -c copy {mkv_name}")


extract_mp3_from_mp4(video_file_name, audio_file_name)
convert_to_mp4(video_file_name)

audio_file = genai.upload_file(path=audio_file_name)

In [11]:
class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  if not file.endswith('.jpg'):
    continue
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
uploaded_files = []
print(f'Uploading {len(files_to_upload)} files. This might take a bit...')

for file in tqdm(files_to_upload):
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")
     

Uploading 285 files. This might take a bit...


  0%|          | 0/285 [00:00<?, ?it/s]

Uploading: frames/intros_mp4_frame00:00.jpg...


  0%|          | 1/285 [00:01<05:01,  1.06s/it]

Uploading: frames/intros_mp4_frame00:01.jpg...


...

Uploading: frames/intros_mp4_frame04:43.jpg...


100%|█████████▉| 284/285 [05:41<00:01,  1.06s/it]

Uploading: frames/intros_mp4_frame04:44.jpg...


100%|██████████| 285/285 [05:42<00:00,  1.20s/it]

Completed file uploads!

Uploaded: 285 files


In [12]:
# List files uploaded in the API
for n, f in zip(range(len(uploaded_files)), genai.list_files()):
  print(f.uri)

https://generativelanguage.googleapis.com/v1beta/files/22f04sjdq4nu
https://generativelanguage.googleapis.com/v1beta/files/sgygcrbtjea9
https://generativelanguage.googleapis.com/v1beta/files/lyita20l8tua
https://generativelanguage.googleapis.com/v1beta/files/d5z9m8grt2fa
https://generativelanguage.googleapis.com/v1beta/files/n2w13bp33qe7
https://generativelanguage.googleapis.com/v1beta/files/wcieukm9r2gs
https://generativelanguage.googleapis.com/v1beta/files/md2qf4q8unc9
https://generativelanguage.googleapis.com/v1beta/files/tigm183692ha
https://generativelanguage.googleapis.com/v1beta/files/eqjue9iowo4h
https://generativelanguage.googleapis.com/v1beta/files/igar9ixxpyat
https://generativelanguage.googleapis.com/v1beta/files/g3xv8v7auwu3
https://generativelanguage.googleapis.com/v1beta/files/6k6he2mjo8au
https://generativelanguage.googleapis.com/v1beta/files/p531bq093xr8
https://generativelanguage.googleapis.com/v1beta/files/3gaqcppnmgkt
https://generativelanguage.googleapis.com/v1beta

In [13]:

#Choose only clips that are entertaining or make for good b-roll.
prompt = """
You are an expert video editor, helping us select clips for a video.
Select only clips where someone tells us their name.
Double check your work to ensure that the timestamps are correct.

For each clip, provide a "start_timestamp", "end_timestamp", "location" (point of interest in the frame, either left, middle-left, middle, middle-right, or right), and "description" (a brief explanation of why you selected the clip).
"""

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest", generation_config={"response_mime_type": "application/json"})

# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    print(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files) + [audio_file]
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

00:00
00:01
00:02
00:03
00:04
00:05
00:06
00:07
00:08
00:09
00:10
00:11
00:12
00:13
00:14
00:15
00:16
00:17
00:18
00:19
00:20
00:21
00:22
00:23
00:24
00:25
00:26
00:27
00:28
00:29
00:30
00:31
00:32
00:33
00:34
00:35
00:36
00:37
00:38
00:39
00:40
00:41
00:42
00:43
00:44
00:45
00:46
00:47
00:48
00:49
00:50
00:51
00:52
00:53
00:54
00:55
00:56
00:57
00:58
00:59
01:00
01:01
01:02
01:03
01:04
01:05
01:06
01:07
01:08
01:09
01:10
01:11
01:12
01:13
01:14
01:15
01:16
01:17
01:18
01:19
01:20
01:21
01:22
01:23
01:24
01:25
01:26
01:27
01:28
01:29
01:30
01:31
01:32
01:33
01:34
01:35
01:36
01:37
01:38
01:39
01:40
01:41
01:42
01:43
01:44
01:45
01:46
01:47
01:48
01:49
01:50
01:51
01:52
01:53
01:54
01:55
01:56
01:57
01:58
01:59
02:00
02:01
02:02
02:03
02:04
02:05
02:06
02:07
02:08
02:09
02:10
02:11
02:12
02:13
02:14
02:15
02:16
02:17
02:18
02:19
02:20
02:21
02:22
02:23
02:24
02:25
02:26
02:27
02:28
02:29
02:30
02:31
02:32
02:33
02:34
02:35
02:36
02:37
02:38
02:39
02:40
02:41
02:42
02:43
02:44
02:45
02:4

In [71]:
from json import loads

clip_details = loads(response.text)

def get_seconds(time_string):
    denoms = time_string.split(":")
    secs = 0
    for i, val in enumerate(denoms[::-1]):
        secs += float(val) * 60 ** i 
    return secs

def clip(start, end, mkv_filename, filename):
    call_ffmpeg(f"-ss {start} -i {mkv_filename} -c copy -t {end - start} {filename}")

for i, details in enumerate(clip_details, start=1):
    clip_name = f"frames/clip_{i}.mkv"
    clip(get_seconds(details['start_timestamp']), get_seconds(details['end_timestamp']), video_file_name, clip_name)

In [ ]:
#print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
#for file in uploaded_files:
#  genai.delete_file(file.response.name)
#  print(f'Deleted {file.file_path} at URI {file.response.uri}')
#print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")

In [37]:
import os

frames_directory = 'frames'
total_size = 0

for root, dirs, files in os.walk(frames_directory):
    for file in files:
        file_path = os.path.join(root, file)
        total_size += os.path.getsize(file_path)

print(f"The total size of all files in the frames directory is: {total_size} bytes")


The total size of all files in the frames directory is: 139088916 bytes
